In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime 

In [4]:
def extract(url, table_attribs):
    ''' This function extracts the required
    information from the website and saves it to a dataframe. The
    function returns the dataframe for further processing. '''

    return df

def transform(df):
    ''' This function converts the GDP information from Currency
    format to float value, transforms the information of GDP from
    USD (Millions) to USD (Billions) rounding to 2 decimal places.
    The function returns the transformed dataframe.'''

    return df

def load_to_csv(df, csv_path):
    ''' This function saves the final dataframe as a `CSV` file 
    in the provided path. Function returns nothing.'''

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final dataframe as a database table
    with the provided name. Function returns nothing.'''

def run_query(query_statement, sql_connection):
    ''' This function runs the stated query on the database table and
    prints the output on the terminal. Function returns nothing. '''

def log_progress(message):
    ''' This function logs the mentioned message at a given stage of the code execution to a log file. Function returns nothing'''


' Here, you define the required entities and call the relevant \nfunctions in the correct order to complete the project. Note that this\nportion is not inside any function.'

In [5]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

In [6]:
#Extract the web page as text
html_page = requests.get(url).text

In [7]:
#Parse the html page    
data = BeautifulSoup(html_page, 'html.parser')

In [9]:
#Create an empty pandas DataFrame named df with columns as the table_attribs
df = pd.DataFrame(columns=table_attribs)

In [11]:
#Extract all 'tbody' attributes of the HTML object and then extract all the rows of the index 2 table using the 'tr' attribute.
tables = data.find_all('tbody')

<tbody><tr>
<td><div class="timeline-wrapper"><map name="timeline_m70l6x3u1y0sgrhu74gnau5009qo98z"></map><img src="//web.archive.org/web/20230902185326im_/https://upload.wikimedia.org/wikipedia/en/timeline/m70l6x3u1y0sgrhu74gnau5009qo98z.png" usemap="#timeline_m70l6x3u1y0sgrhu74gnau5009qo98z"/></div>
</td></tr>
<tr>
<td style="text-align:center; font-size:90%;">Largest economies in the world by GDP (nominal) in 2023<br/>according to <a href="/web/20230902185326/https://en.wikipedia.org/wiki/International_Monetary_Fund" title="International Monetary Fund">International Monetary Fund</a> estimates<sup class="reference" id="cite_ref-China-THM_1-0"><a href="#cite_note-China-THM-1">[n 1]</a></sup><sup class="reference" id="cite_ref-GDP_IMF_2-0"><a href="#cite_note-GDP_IMF-2">[1]</a></sup>
</td></tr></tbody>
